In [ ]:
from tqdm import tqdm
import pandas as pd
import string
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from sentence_transformers import SentenceTransformer
import pickle
import re
import itertools
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:

data_filter_repolist_with_full_info = pd.read_csv("../data/data_filter_repolist_with_full_info_no_header.csv",header=None)

SATD_comment_sum = pd.DataFrame()

for index, row in tqdm(data_filter_repolist_with_full_info.iterrows(), total=data_filter_repolist_with_full_info.shape[0]):
    comment = pd.read_csv('../data/SATD_comment/' + str(row[0]) + '.csv')
    SATD_comment_sum = pd.concat([SATD_comment_sum, comment], ignore_index=True)


In [ ]:
SATD_comment_sum['filePath'] = SATD_comment_sum['linkLocation'].apply(lambda x: ''.join(x.split('#L')[:-1]))

In [ ]:
Non_SATD_comment_sum = pd.DataFrame()

for index in tqdm(SATD_comment_sum['repoIndex'].unique()):
    comment = pd.read_csv('../data/Comments_with_no_keywords/' + str(index) + '.csv')
    Non_SATD_comment_sum = pd.concat([Non_SATD_comment_sum, comment], ignore_index=True)


In [ ]:
Non_SATD_comment_sum['filePath'] = Non_SATD_comment_sum['linkLocation'].apply(lambda x: ''.join(x.split('#L')[:-1]))

In [ ]:
len(Non_SATD_comment_sum['repoIndex'].unique())

In [ ]:
Non_SATD_comment_sum['buildSysteminFile'].value_counts()

In [ ]:
Non_SATD_comment_sum_same_file = Non_SATD_comment_sum.loc[Non_SATD_comment_sum['filePath'].isin(list(SATD_comment_sum['filePath'].unique()))]

In [ ]:
Non_SATD_comment_sum_sample = pd.DataFrame(columns=Non_SATD_comment_sum_same_file.columns, dtype=object)

unexptected_case = {}
for index, row in SATD_comment_sum.iterrows():
    comments = Non_SATD_comment_sum_same_file.loc[(Non_SATD_comment_sum_same_file['filePath'] == row['filePath']) & (~Non_SATD_comment_sum_same_file.index.isin(list(Non_SATD_comment_sum_sample.index)))]
    if comments.shape[0] == 0:
        if row['filePath'] in unexptected_case.keys():
            unexptected_case[row['filePath']] += 1
        else: unexptected_case[row['filePath']] = 0
        continue
    line_diff_upper = [line for line in list(comments['startLine']) if row['startLine'] > line]
    indexes = []
    if len(line_diff_upper) != 0:
        indexes.append(line_diff_upper.index(max(line_diff_upper)))
    line_diff_below = [line for line in list(comments['endLine']) if row['endLine'] < line]
    if len(line_diff_below) != 0:
        indexes.append(line_diff_below.index(min(line_diff_below)))
    for i in indexes:
        Non_SATD_comment_sum_sample = Non_SATD_comment_sum_sample.append(comments.iloc[i])


In [ ]:
df = Non_SATD_comment_sum_sample.drop_duplicates()

In [ ]:
df['buildSysteminFile'].value_counts()

In [ ]:
len(df['repoIndex'].unique())

In [ ]:
import re
def clean_url(text):
    text = ''.join([word if word not in string.punctuation else ' ' for word in text.group(0)])
    return text

def clean_string(text):
    text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)',clean_url, text)
    text = re.sub(r'\s+',' ', text)
    if len(re.findall(r'\bdnl\b', text)) > 0:
        lines = []
        for line in text.split('\n'):
            if len(line.split()) > 0:
                if 'dnl' == line.split()[0]:
                    line = line.replace('dnl', '', 1)
            lines.append(line)
        text ='\n'.join(lines)
    text = re.sub(r'[^A-Za-z0-9.\']+',' ',text)
    text = text.lower()
    text = text.strip()
    return text

In [ ]:
cleaned_comments = []
Non_SATD_comment_sample_remove_single_word = pd.DataFrame(columns = df.columns, dtype=object )
for index, row in df.iterrows():
    cleaned_comment = clean_string(row['comment'])
    word_limit = 2
    if len(cleaned_comment.split()) >= word_limit:
        cleaned_comments.append(cleaned_comment)
        Non_SATD_comment_sample_remove_single_word = Non_SATD_comment_sample_remove_single_word.append(row, ignore_index=True)
Non_SATD_comment_sample_remove_single_word['cleaned_comments'] = cleaned_comments



In [ ]:
Non_SATD_comment_sample_remove_single_word['buildSysteminFile'].value_counts()

In [ ]:
len(Non_SATD_comment_sample_remove_single_word['repoName'].unique())

In [ ]:
comments_embeddings = model.encode(list(Non_SATD_comment_sample_remove_single_word['cleaned_comments']))

In [ ]:
csim = cosine_similarity(comments_embeddings)

In [ ]:
less_than_0_8_indexes = np.where(np.count_nonzero(csim >= 0.8, axis=1) <= 1)
comments_embeddings = np.delete(comments_embeddings,less_than_0_8_indexes[0],axis = 0 )
Non_SATD_comment_sample_remove_single_word_and_less_0_8 = Non_SATD_comment_sample_remove_single_word.drop(less_than_0_8_indexes[0])
Non_SATD_comment_sample_remove_single_word_and_less_0_8.reset_index(drop=True, inplace=True)
len(Non_SATD_comment_sample_remove_single_word_and_less_0_8['repoName'].unique())

In [ ]:
Non_SATD_comment_sample_remove_single_word_and_less_0_8['buildSysteminFile'].value_counts()

In [ ]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=0.1, min_samples=2, n_jobs = -1, metric='cosine').fit_predict(comments_embeddings)

In [ ]:
Non_SATD_comment_sample_remove_single_word_and_less_0_8['groupId'] = clustering

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(comments_embeddings, clustering, metric='cosine')

In [ ]:
Non_SATD_comment_sample_remove_single_word_and_less_0_8_and_noise = Non_SATD_comment_sample_remove_single_word_and_less_0_8.loc[Non_SATD_comment_sample_remove_single_word_and_less_0_8['groupId'] != -1]

In [ ]:
len(Non_SATD_comment_sample_remove_single_word_and_less_0_8_and_noise['repoName'].unique())

In [ ]:
Non_SATD_comment_sample_remove_single_word_and_less_0_8_and_noise['buildSysteminFile'].value_counts()

In [ ]:
Non_SATD_comments = pd.DataFrame()

for name, group in Non_SATD_comment_sample_remove_single_word_and_less_0_8_and_noise.groupby('groupId'):
    repo_diversity = len(set(list(group['repoName'])))
    group['repoDiversity'] = [repo_diversity] * group.shape[0]
    build_system_set = set(list(group['buildSysteminFile']))
    if set(['CMake','Autotool']).intersection(build_system_set) and set(['Maven','Ant']).intersection(build_system_set):
        group['isCrossLanguage'] = [True] * group.shape[0]
    else:
        group['isCrossLanguage'] = [False] * group.shape[0]
    system_diversity = len(build_system_set)
    group['systemDiversity'] = [system_diversity] * group.shape[0]
    Non_SATD_comments = pd.concat([Non_SATD_comments,group])

        

In [ ]:
Non_SATD_groups = pd.DataFrame({'count' : Non_SATD_comments.groupby(['groupId','repoDiversity','isCrossLanguage','systemDiversity']).size()}).reset_index()

In [ ]:
Non_SATD_groups.loc[Non_SATD_groups['repoDiversity'] == 1]['count'].describe()

In [ ]:
sum(Non_SATD_groups.loc[Non_SATD_groups['repoDiversity'] == 1]['count'])

In [ ]:
Non_SATD_groups.loc[Non_SATD_groups['repoDiversity'] != 1]['count'].describe()

In [ ]:
sum(Non_SATD_groups.loc[Non_SATD_groups['repoDiversity'] != 1]['count'])

In [ ]:
Non_SATD_groups.loc[Non_SATD_groups['isCrossLanguage'] == False]['count'].describe()

In [ ]:
sum(Non_SATD_groups.loc[Non_SATD_groups['isCrossLanguage'] == False]['count'])

In [ ]:
Non_SATD_groups.loc[Non_SATD_groups['isCrossLanguage'] == True]['count'].describe()

In [ ]:
sum(Non_SATD_groups.loc[Non_SATD_groups['isCrossLanguage'] == True]['count'])

In [ ]:
Non_SATD_groups.loc[Non_SATD_groups['systemDiversity'] == 1]['count'].describe()

In [ ]:
sum(Non_SATD_groups.loc[Non_SATD_groups['systemDiversity'] == 1]['count'])

In [ ]:
Non_SATD_groups.loc[Non_SATD_groups['systemDiversity'] != 1]['count'].describe()

In [ ]:
sum(Non_SATD_groups.loc[Non_SATD_groups['systemDiversity'] != 1]['count'])

In [ ]:
Non_SATD_comments[Non_SATD_comments['systemDiversity'] == 1]['buildSysteminFile'].value_counts()

In [ ]:
Non_SATD_comments[Non_SATD_comments['systemDiversity'] == 1].groupby(['buildSysteminFile'])['groupId'].agg(lambda x: len(set(x)))

In [ ]:
from collections import Counter

same_system_clone = Non_SATD_comments[Non_SATD_comments['systemDiversity'] == 1].groupby(['buildSysteminFile'])['groupId'].agg(lambda x: {key:value for key, value in dict(Counter(x)).items()}).to_frame()

same_system_clone_Ant = pd.DataFrame(same_system_clone.iloc[0]['groupId'].values(),
                                              columns =['values'])
same_system_clone_Ant['values'].describe()

In [ ]:
same_system_clone_Autotool = pd.DataFrame(same_system_clone.iloc[1]['groupId'].values(),
                                              columns =['values'])
same_system_clone_Autotool['values'].describe()

In [ ]:
same_system_clone_CMake = pd.DataFrame(same_system_clone.iloc[2]['groupId'].values(),
                                              columns =['values'])
same_system_clone_CMake['values'].describe()

In [ ]:
same_system_clone_Maven = pd.DataFrame(same_system_clone.iloc[3]['groupId'].values(),
                                              columns =['values'])
same_system_clone_Maven['values'].describe()

In [ ]:
Non_SATD_comments.to_csv('../data/Non_SATD_clones.csv',index=None)